In [137]:
import csv
import glob
from os import listdir
import os
from xml.etree import ElementTree
from os.path import isfile, join 
from bs4 import BeautifulSoup 
from nltk.tokenize import RegexpTokenizer 
import pandas as pd
import numpy as np 
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import networkx as nx 
from pathlib import Path
from sklearn import preprocessing
import sys
import pickle as pkl
from scipy import sparse as sp
from sklearn.decomposition import TruncatedSVD
from typing import *
import copy
import os
from sklearn import metrics  as m
from plotly.subplots import make_subplots
import numpy as np
import plotly.graph_objects as go
import plotly
import networkx as nx
import scipy.sparse as sp
import scipy as sc
import scipy as sc
import time
import pandas as pd
from genpr.tensorflow.io_planetoid_type import load_data
from genpr.data.io import load_dataset
import tqdm
from sklearn.preprocessing import normalize
from scipy.spatial import distance
from sklearn.metrics import average_precision_score, accuracy_score
from sklearn.neighbors import NearestNeighbors
from scipy.sparse.linalg import gmres
import timeit
from scipy import sparse
from sklearn.decomposition import TruncatedSVD

## Processing lxml files

### Data Available by link 
-- https://clinicaltrials.gov/ct2/download_studies?term=COVID&down_chunk=2

In [93]:
entries = os.listdir('covid/')

- Extracting description of clinical studies:  
    1. ''official\_title''
    2. ''brief\_summary''
    3. ''detailed\_description''
    4.  ''eligibility''  

- Labels:
    1. ''masking'' 
 

In [71]:
text_ = []
class_= []
for i in entries:
    tt = ""
    with open('covid/'+i, "r") as file: 
        content = file.readlines() 
        content = "".join(content)
        bs_content = BeautifulSoup(content, "lxml")
        if bs_content.find("official_title" )!= None :
            third_child = bs_content.find("official_title" )
            tt+= list( list(third_child.children))[0]
        if bs_content.find("brief_summary" )!= None : 
            third_child = bs_content.find("brief_summary" )
            tt+= ' '.join( list(third_child.strings) ).replace('\n', '')
        if bs_content.find("detailed_description" )!= None :
            third_child = bs_content.find("detailed_description" ) 
            tt+=' '.join( list(third_child.strings) ).replace('\n', '')
        if bs_content.find("eligibility" )!= None :
            third_child = bs_content.find("eligibility" ) 
            tt+= ' '.join( list(third_child.strings) ).replace('\n', '')
        if bs_content.find("masking" ) != None:
            class_.append(bs_content.find("masking" ).text)
        else:
            class_.append(0)
        text_.append(tt) 
            
            


In [72]:
ntext = []
for t in class_: 
    if t != 0:
        tt = t.split(' ')[0]
        if tt == 'None':
            tt = t.split(' ')[1].replace("(","")
        ntext.append(tt)
    else:
        ntext.append(0)

In [73]:
class_[1].split(' ')[0]

'None'

- Filter NaN masks

In [74]:
d = {'text': text_, 'class':ntext}
df = pd.DataFrame(data=d)

In [75]:
df_free = df[df['class'] != 0]

- Replace Not Open mask by Blind

In [94]:
df_free["class"].replace({"Double": "Blind", 
                          "Quadruple": "Blind",
                          "Single":"Blind", 
                          "Triple":"Blind", }, inplace=True)

## Processing dataset

In [77]:
vectorizer = CountVectorizer( stop_words='english', analyzer='word', min_df=5,)
X = vectorizer.fit_transform(df_free['text']) 

In [88]:
le = preprocessing.LabelEncoder()
Y = le.fit_transform(df_free['class'])

In [95]:
neigh = NearestNeighbors(n_neighbors=7, metric='dice', )
neigh.fit(X.toarray())
distances, indices = neigh.kneighbors(X.toarray())
Af = np.zeros((X.shape[0], X.shape[0]))
for i in tqdm.tqdm(range(X.shape[0])):
    Af[i, indices[i]] = 1
    Af[ indices[i], i] = 1

100%|██████████| 2039/2039 [00:00<00:00, 177814.91it/s]


## Splitting data as in 
## Revisiting Semi-Supervised Learning with Graph Embeddings 

In [96]:
def exclude_idx(idx: np.ndarray, idx_exclude_list: List[np.ndarray]) -> np.ndarray:
    idx_exclude = np.concatenate(idx_exclude_list)
    return np.array([i for i in idx if i not in idx_exclude])
  
def known_unknown_split(idx: np.ndarray, nknown, seed) -> Tuple[np.ndarray, np.ndarray]:
    rnd_state = np.random.RandomState(seed)
    known_idx = rnd_state.choice(idx, nknown, replace=False)
    unknown_idx = exclude_idx(idx, [known_idx])
    return known_idx, unknown_idx
  
def train_stopping_split( idx: np.ndarray, labels: np.ndarray, ntrain_per_class, nstopping, seed, ) -> Tuple[np.ndarray, np.ndarray]:
    rnd_state = np.random.RandomState(seed)
    train_idx_split = []
    for i in np.unique(labels):
        train_idx_split.append(
            rnd_state.choice(idx[labels == i], ntrain_per_class, replace=False)
        )
    train_idx = np.concatenate(train_idx_split)
    stopping_idx = rnd_state.choice(
        exclude_idx(idx, [train_idx]), nstopping, replace=False
    )
    return train_idx, stopping_idx
  
def gen_splits(labels: np.ndarray, idx_split_args: Dict[str, int], test ) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    all_idx = np.arange(len(labels))
    known_idx, unknown_idx = known_unknown_split(
        all_idx, idx_split_args["nknown"], idx_split_args["seed"]
    )
    _, cnts = np.unique(labels[known_idx], return_counts=True)
    stopping_split_args = copy.copy(idx_split_args)
    del stopping_split_args["nknown"]
    train_idx, stopping_idx = train_stopping_split(
        known_idx, labels[known_idx], **stopping_split_args
    )
    if test:
        val_idx = unknown_idx
    else:
        val_idx = exclude_idx(known_idx, [train_idx, stopping_idx])
    return train_idx, stopping_idx, val_idx

In [97]:
idx_split_args = {
    "ntrain_per_class": 20,
    "nstopping": 500,
    "nknown": 1362,
    "seed": 0,
}
 
train_idx, _, test_idx = gen_splits(
    labels=Y, idx_split_args=idx_split_args, test=True
)

## Processing data for training

In [138]:
Xm = X.toarray().T
Xm = Xm - np.median(Xm, axis=0)
MMx = np.dot(Xm.T,Xm ) /(Xm.shape[0]-1)

In [112]:
new_labels = train_idx 
y_all = np.zeros((X.shape[0], 2))
for i in new_labels: 
    c_ = int(Y[i] )  
    y_all[i, c_] = 1

In [113]:
def calc_A_hat(adj_matrix: sp.spmatrix, delta, sigma, MMx) -> sp.spmatrix:
    nnodes = adj_matrix.shape[0]
    A = adj_matrix + sp.eye(nnodes)#Ω#@ D_invsqrt_corr
    D_vec = np.sum(A, axis=1).A1 
    lsigma = sigma -1
    rsigma = - sigma
    wsigma = -2*sigma + 1
    
    D_l = sp.diags(np.power(D_vec, lsigma)) 
    D_r= sp.diags(np.power(D_vec, rsigma ) )
    Dw = sp.diags(np.power(D_vec, wsigma ) )
    
    
    return  D_l@A@D_r  +   delta*  MMx@ Dw 

### Cosinuse and RBF distances

In [140]:
MMCos = cosine_similarity(X) /(X.shape[1]-1) 
MMrbf = rbf_kernel(X) /(X.shape[1] -1)

In [141]:
def calc_A_hat(adj_matrix: sp.spmatrix, delta, sigma, MMx) -> sp.spmatrix:
    nnodes = adj_matrix.shape[0]
    A = adj_matrix + sp.eye(nnodes) 
    D_vec = np.sum(A, axis=1).A1 
    lsigma = sigma - 1
    rsigma = - sigma
    wsigma = -2*sigma + 1
    
    D_l = sp.diags(np.power(D_vec, lsigma)) 
    D_r= sp.diags(np.power(D_vec, rsigma ) )
    Dw = sp.diags(np.power(D_vec, wsigma ) )
    S_ = MMx@ Dw 
    
    return S_ , D_l@A@D_r  + delta* S_

svd_error = 0.1 
delta = 1
sigma = 1
alpha= 0.9
svd = TruncatedSVD(n_components=1, algorithm='randomized')
mmc, AHAT = calc_A_hat(Af, delta, sigma, MMx ) 
rex = (np.identity(nnodes)  - alpha * AHAT )
svd.fit(mmc)
gamma = svd.singular_values_[0]

def JOR(A, Z, Y, iter_, beta, alpha):
    A = np.copy(A)
    Z = np.copy(Z)
    Y = np.copy(Y) 
    for _ in range(iter_):
        Z = Z + beta*(alpha * AHAT@Z - Z + (1-alpha) * Y) 
    return Z

def GMRES(A, Y, alpha, k, tol):
    A = np.copy(A)
    Y = np.copy(Y) 
    predicts = []
    for j in range(k): 
        temp_ = sparse.linalg.gmres(A, (1-alpha)*Y[:,j], tol=tol)[0] 
        predicts.append([temp_])
    return np.concatenate(predicts).T 

if (1 + delta* (gamma - svd_error)) <= 1/alpha: 
    print('jor')
    start = timeit.default_timer()
    Z = JOR(A=AHAT, Z=y_all, Y=y_all, iter_=15, beta=0.9, alpha=alpha)
    print(timeit.default_timer() - start)
else:
    print('gmres')
    start = timeit.default_timer()
    Z = GMRES(A=rex, Y=y_all, alpha=alpha, k=y_all.shape[1], tol=1e-03)
    print(timeit.default_timer() - start)
    
m.f1_score( Y[test_idx] ,
                  np.argmax(np.array(Z  )[test_idx], axis=-1), average='micro') 

gmres
0.12949191700317897


0.6115214180206795

### Compute Label Propagation

In [136]:
from sklearn.semi_supervised import LabelPropagation
Ytr = np.zeros(Y.shape) -1
Ytr[train_idx] = Y[train_idx]
label_prop_model = LabelPropagation(kernel='rbf')
label_prop_model.fit(X.toarray(), Ytr )
Z = label_prop_model.predict(X.toarray())
accuracy_score( Y[test_idx], Z[test_idx] )

/Users/mikhail/opt/anaconda3/lib/python3.7/site-packages/sklearn/semi_supervised/_label_propagation.py:277: RuntimeWarning:

invalid value encountered in true_divide

/Users/mikhail/opt/anaconda3/lib/python3.7/site-packages/sklearn/semi_supervised/_label_propagation.py:288: ConvergenceWarning:

max_iter=1000 was reached without convergence.



0.465288035450517